In [165]:
from efficientnet_pytorch import EfficientNet
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.optim import lr_scheduler
from torchvision import transforms, models
import os
import copy

In [166]:
class DataLoaderOil(data.Dataset):
    """
        Dataset class for semantic segmentation tasks.
    """
    def __init__(self, folder_path):
        super(DataLoaderOil, self).__init__()

        self.npy_files  = []
        for filename in os.listdir(os.path.join(folder_path, 'pipes_frames')):
            ext=os.path.splitext(filename)[1]
            if ext not in ['.db']:
                self.npy_files.append(os.path.join(folder_path, 'pipes_frames', filename))
        
        for filename in os.listdir(os.path.join(folder_path, 'spills_frames')):
            ext=os.path.splitext(filename)[1]
            if ext not in ['.db']:
                self.npy_files.append(os.path.join(folder_path, 'spills_frames', filename))
                
        
        print(len(self.npy_files))
        
        self.start_transform = transforms.Compose([transforms.ToTensor()])
        self.normalize_transform =  transforms.Compose([transforms.Normalize([0],[1])])

    def __getitem__(self, index):
        npy_path = self.npy_files[index]
        data = np.load(npy_path)
        label = 1 if npy_path.find('spills_frames') != -1 else 0
        
        
        
        
        transform_data = self.start_transform(data)
        transform_label = label

        transform_data = self.normalize_transform(transform_data)
        return transform_data, transform_label
    
    def __len__(self):
        return len(self.npy_files)

In [167]:
DATA_DIR = './dataset'
npy_dataset =DataLoaderOil(DATA_DIR)

78749


In [168]:
valid_size = 0.2
num_train = len(npy_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
np.random.shuffle(indices)
from torch.utils.data.sampler import SubsetRandomSampler
train_idx, test_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)
trainloader = torch.utils.data.DataLoader(npy_dataset, sampler=train_sampler, batch_size=4)
testloader = torch.utils.data.DataLoader(npy_dataset, sampler=test_sampler, batch_size=4)

In [169]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [170]:
import torch.nn as nn
import torch.nn.functional as F

In [202]:
model = models.mobilenet_v3_small(pretrained=True)
# model.fc = nn.Linear(2048, 1)

In [203]:
model.features[0][0] = nn.Conv2d(19, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

In [204]:
model.classifier[3] = nn.Linear(in_features=1024, out_features=1, bias=True)

In [205]:
# model.conv1 = nn.Conv2d(19, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [206]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):

        ce_loss = F.cross_entropy(input, target,reduction=self.reduction,weight=self.weight) 
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

optimizer= optim.Adamax([
    {'params' : model.parameters()},
#     {'params' : model.backbone.layer4.parameters()}
    ]
    , lr=0.0003)
# optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
# criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(full_weights_list).to('cuda'))
# criterion=FocalLoss(weight = torch.Tensor([1, 1]).to('cuda'))
criterion = nn.BCEWithLogitsLoss()
model.to(device);

In [207]:
batch = next(iter(trainloader))

In [208]:
len(batch)

2

In [209]:
batch[0].shape

torch.Size([4, 19, 64, 64])

In [210]:
batch[1]

tensor([1, 1, 1, 1])

In [211]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [218]:
epochs = 10
steps = 0
best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
running_loss = 0
test_acc = 0
accuracy = 0
train_acc = 0
print_every = 100
train_losses, test_losses = [], []
for epoch in range(epochs):
    print(epoch)
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
#         print(logps.shape, labels.shape)
        loss = criterion(logps, labels.float().unsqueeze(1))
        accuracy = binary_acc(logps, labels.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        train_acc += accuracy.item()
        running_loss += loss.item()
    
    
    if steps%10000 == 0:
        print(steps)
    test_acc = 0
    test_loss = 0
    
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)

            batch_loss = criterion(logps, labels.float().unsqueeze(1))
            accuracy = binary_acc(logps, labels.unsqueeze(1))

            test_loss += loss.item()
            test_acc += accuracy.item()
            if test_acc > best_acc:
                best_acc = test_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    train_losses.append(running_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))                    
    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {running_loss/len(trainloader):.3f}.. "
          f"Train accuracy: {train_acc/len(trainloader):.3f}.. "
          f"Test loss: {test_loss/len(testloader):.3f}.. "
          f"Test accuracy: {test_acc/len(testloader):.3f}")
    running_loss = 0
    train_acc = 0
    model.train()   
        
PATH = f"./oil_classify_net_{best_acc:.3f}.pth"
torch.save(best_model_wts, PATH)

0
Epoch 1/10.. Train loss: 0.385.. Train accuracy: 82.089.. Test loss: 0.237.. Test accuracy: 84.345
1
Epoch 2/10.. Train loss: 0.322.. Train accuracy: 85.730.. Test loss: 0.024.. Test accuracy: 78.187
2
Epoch 3/10.. Train loss: 0.276.. Train accuracy: 88.113.. Test loss: 0.066.. Test accuracy: 85.196
3
Epoch 4/10.. Train loss: 0.240.. Train accuracy: 89.965.. Test loss: 0.146.. Test accuracy: 87.195
4
Epoch 5/10.. Train loss: 0.210.. Train accuracy: 91.373.. Test loss: 0.431.. Test accuracy: 90.573
5
Epoch 6/10.. Train loss: 0.190.. Train accuracy: 92.179.. Test loss: 0.016.. Test accuracy: 90.300
6
Epoch 7/10.. Train loss: 0.172.. Train accuracy: 93.016.. Test loss: 0.041.. Test accuracy: 92.382
7
Epoch 8/10.. Train loss: 0.158.. Train accuracy: 93.602.. Test loss: 0.005.. Test accuracy: 92.553
8
Epoch 9/10.. Train loss: 0.147.. Train accuracy: 94.121.. Test loss: 0.121.. Test accuracy: 93.074
9
Epoch 10/10.. Train loss: 0.134.. Train accuracy: 94.503.. Test loss: 0.004.. Test accura